In [ ]:
%load_ext autotime
%matplotlib inline

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
directory = './'

In [ ]:
from find_questions import *

In [ ]:
data = read_dataset(directory + 'clickhouse_ru.labelled.combined.csv')

data.shape

In [ ]:
dialogs_numbers = set(data[data['is_root'] == 1]['dialog_number'])
data = data[data['dialog_number'].isin(dialogs_numbers)]
data.shape

Define which messages should label

In [ ]:
def set_should_predict(df):
    labels = set(df[df['is_root'] == 1]['dialog_number'])
    print('labels', len(labels))

    df_labeled = df[(df['dialog_number'].isin(labels))]
    print('Nodes in dialogs', df_labeled.shape[0])

    df_to_predict = df_labeled[((df_labeled['is_root'] == 0) & (df_labeled['reply_to_msg_id'].isnull()))]
    print('Nodes in predict', df_to_predict.shape[0])

    df['should_predict'] = 0
    df.loc[df['id'].isin(df_to_predict['id']), 'should_predict'] = 1
    
    
set_should_predict(data)
data['should_predict'].value_counts()

Create dataset

In [ ]:
from ReplyTree import *

In [ ]:
original = data

msg_id_to_from_id_dict = {row['id']:row['from_id'] for _, row in original.iterrows()}
msg_id_to_date_dict = {row['id']: row['date_'] for _, row in original.iterrows()}
msg_id_to_dialog_number_dict = {row['id']:row['dialog_number'] for _, row in original.iterrows()}

In [ ]:
def add_from_ids(msg_ids, msg_id_to_from_id_dict):
    return set(msg_id_to_from_id_dict[id_] for id_ in msg_ids)


def add_min_date(msg_ids, msg_id_to_date_dict):
    return min(msg_id_to_date_dict[id_] for id_ in msg_ids)


def add_max_date(msg_ids, msg_id_to_date_dict):
    return max(msg_id_to_date_dict[id_] for id_ in msg_ids)


def add_dialog_number_label(msg_ids1, msg_ids2, msg_id_to_dialog_number_dict):
    return int(msg_id_to_dialog_number_dict[msg_ids1[0]] == msg_id_to_dialog_number_dict[msg_ids2[0]])


def count_msgs_between(msg_ids1, msg_ids2):
    ids1 = list(int(id_) for id1 in msg_ids1 for id_ in id1.split('__'))
    ids2 = list(int(id_) for id2 in msg_ids2 for id_ in id2.split('__'))
    res = 99999
    for id1 in ids1:
        for id2 in ids2:
#             assert id2 > id1
            if res > abs(id2 - id1):
                res = abs(id2 - id1)
    return res


def count_seconds_between(msg_ids1, msg_ids2, msg_id_to_date_dict):
#     ids1 = list(int(id_) for id1 in msg_ids1 for id_ in id1.split('__'))
#     ids2 = list(int(id_) for id2 in msg_ids2 for id_ in id2.split('__'))
#     ids1 = [int(id_) for id_ in msg_ids1]
#     ids2 = [int(id_) for id_ in msg_ids2]
    res = 999999999999999
    for id1 in msg_ids1:
        for id2 in msg_ids2:
#             assert id2 > id1
            time_span = abs(msg_id_to_date_dict[id2] - msg_id_to_date_dict[id1]).total_seconds()
            if res > time_span:
                res = time_span
    return res

In [ ]:
dt = split_into_discussion_trees(data)
print("Dialogs count:", len(dt.trees))
# dt.filter_trees_with_len_1()
# print("Dialogs with len > 1 number:", len(dt.trees))

In [ ]:
for tree in dt.trees:
    print(tree)

In [ ]:
res = {'msg_ids1': [],
       'msg_ids2': []}


def get_nodes(tree):
    return [id_ for id_ in tree.nodes]


for i, first_tree in enumerate(dt.trees):
#     print(tree)
    for second_tree in dt.trees[:i]:
        res['msg_ids2'].append(get_nodes(first_tree))
        res['msg_ids1'].append(get_nodes(second_tree))
#         print(tree)

res = pd.DataFrame.from_dict(res)
len(res)

In [ ]:
res['from_ids1'] = res['msg_ids1'].apply(lambda msg_ids: add_from_ids(msg_ids, msg_id_to_from_id_dict))
res['from_ids2'] = res['msg_ids2'].apply(lambda msg_ids: add_from_ids(msg_ids, msg_id_to_from_id_dict))

res['min_date1'] = res['msg_ids1'].apply(lambda msg_ids: add_min_date(msg_ids, msg_id_to_date_dict))
res['min_date2'] = res['msg_ids2'].apply(lambda msg_ids: add_min_date(msg_ids, msg_id_to_date_dict))

res['max_date1'] = res['msg_ids1'].apply(lambda msg_ids: add_max_date(msg_ids, msg_id_to_date_dict))
res['max_date2'] = res['msg_ids2'].apply(lambda msg_ids: add_max_date(msg_ids, msg_id_to_date_dict))


res['msgs_between'] = res.apply(
    lambda row: count_msgs_between(row['msg_ids1'], row['msg_ids2']), axis=1)


res['seconds_between'] = res.apply(
    lambda row: count_seconds_between(row['msg_ids1'], row['msg_ids2'], msg_id_to_date_dict), 
    axis=1)


res['equal_dialog_number'] = res.apply(
    lambda row: add_dialog_number_label(row['msg_ids1'], row['msg_ids2'], msg_id_to_dialog_number_dict), 
    axis=1)

In [ ]:
res['label'] = res['equal_dialog_number']

In [ ]:
res['label'].value_counts()

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)

In [ ]:
plotly_data = [
    go.Histogram(
        x=res['msgs_between'],
        name='',
        opacity=0.9
    )]

layout = go.Layout(
    yaxis=dict(title='<b></b>', title_font=dict(size=18)),
    xaxis=dict(title='<b></b>', title_font=dict(size=18)),
    title={'text': '<b></b>',
           'xanchor': 'left',
           'pad': {'l': 240},
           'font': {'size': 18}},
    bargap=0.01)

fig = go.Figure(data=plotly_data, layout=layout)
iplot(fig)

In [ ]:
# res[res['msgs_between'] > 0]
len(res[res['msgs_between'] <= 10])

In [ ]:
from sklearn.utils import shuffle

In [ ]:
def balance(df):
    df0 = df[df['label'] == 0]
    df1 = df[df['label'] == 1]
    min_size = min(df0.shape[0], df1.shape[0])
    return shuffle(pd.concat([df0.sample(min_size), df1.sample(min_size)])).reset_index()

In [ ]:
df = res.reset_index()
# df = res[res['msgs_between'] <= 10].reset_index()
df = balance(df)

df['equal_dialog_number'].value_counts()

In [ ]:
df[df['equal_dialog_number'] == 1]['msgs_between'].value_counts()

In [ ]:
df[df['equal_dialog_number'] == 0]['msgs_between'].value_counts()

In [ ]:
df

In [ ]:
def add_fields(df, version):
    # 2
    df['msg_ids1_number'] = df['msg_ids1'].apply(lambda msg_ids1: len(msg_ids1))
    # 3
    df['msg_ids2_number'] = df['msg_ids2'].apply(lambda msg_ids2: len(msg_ids2))
    # 4
#     df['messages_number_between'] = df.apply(
#         lambda row: calculate_msgs_number_between(row['msg_ids1'], row['msg_ids2'], version), axis=1
#     )
    df['messages_number_between'] = df.apply(
        lambda row: count_msgs_between(row['msg_ids1'], row['msg_ids2']), axis=1
    )
    # 5
    df['from_ids1_number'] = df['from_ids1'].apply(len)
    # 6
    df['from_ids2_number'] = df['from_ids2'].apply(len)
    # 7
    df['from_ids_intersection_number'] = df.apply(
        lambda row: len(set(row['from_ids1']).intersection(set(row['from_ids2']))), axis=1)
    # 8
    df['len_in_seconds1'] = df.apply(
        lambda row: (row['max_date1'] - row['min_date1']).total_seconds(), axis=1)
    # 9
    df['len_in_seconds2'] = df.apply(
        lambda row: (row['max_date2'] - row['min_date2']).total_seconds(), axis=1)
#     # 10
#     df['distance_in_seconds'] = df.apply(
#         lambda row: calculate_min_distance_in_seconds(
#             row['min_date1'], row['max_date1'], row['min_date2'], row['max_date2']), axis=1)
    return df

In [ ]:
add_fields(df, '')

In [ ]:
df.columns

In [ ]:
features_names = ['messages_number_between',
                  'seconds_between', 'msg_ids1_number',
                  'msg_ids2_number', 'from_ids1_number', 'from_ids2_number',
                  'from_ids_intersection_number', 'len_in_seconds1', 'len_in_seconds2',
#                   'bert_p'
                 ]

In [ ]:
df['label'] = df['equal_dialog_number']

In [ ]:
import xgboost as xgb

In [ ]:
def xgb_fabric():
    return xgb.XGBClassifier(n_estimators=100, 
                             subsample=0.6, colsample_bytree=0.6,
                             objective= 'binary:logistic',

                             max_depth=10,
                             learning_rate=0.03,
                             min_child_weight=1,
                             
                             n_jobs=-1)

In [ ]:
from sklearn import svm
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, StratifiedKFold

from metrics import *
from sklearn.metrics import precision_recall_fscore_support

from tqdm.notebook import tqdm

In [ ]:
def cross_val_predict(df, fabric, features=features_names, kf=LeaveOneOut(), n_splits=5):
    metrics_list = []
    df['predict'] = 0
    for train_idx, test_idx in tqdm(kf.split(df, df['label'])):
        X_train, X_test = df[features].loc[train_idx], df[features].loc[test_idx]
        assert X_train.shape[1] == len(features)
        assert X_test.shape[1] == len(features)
        y_train, y_test = df.loc[train_idx, 'label'],  df.loc[test_idx, 'label']

        clf = fabric()
        clf.fit(X_train, y_train)
#         print(X_test.shape, len(test_idx))
        df.loc[test_idx, 'predict'] = clf.predict_proba(X_test)[:, 1]
#         print(df.loc[test_idx, 'predict'])
#         df.loc[test_idx, 'predict'] = clf.predict(X_test)
#     df['predict'] = df['predict'].astype(int)
    metrics = precision_recall_fscore_support(df['label'], df['predict'].apply(round))
    metrics_df = pd.DataFrame.from_dict({'class': [0, 1],
                                         'precision': metrics[0],
                                         'recall': metrics[1],
                                         'f1': metrics[2],
                                         'support': metrics[3]})
    metrics_list.append(metrics_df)
    metrics = pd.concat(metrics_list).groupby('class').mean()
    return df['predict'].copy(), metrics

In [ ]:
df.shape

In [ ]:
df['predict'], metrics = cross_val_predict(df, xgb_fabric, features=features_names, kf=StratifiedKFold(5), n_splits=5)
metrics

### Restore numbers of dialogs from pairs

In [ ]:
df['predict_proba'] = df['predict']
df['predict'] = df['predict'].apply(round)

In [ ]:
df['msg_ids1_0'] = df['msg_ids1'].apply(lambda msg_ids: msg_ids[0])
df['msg_ids2_0'] = df['msg_ids2'].apply(lambda msg_ids: msg_ids[0])

In [ ]:
x =  df.groupby('msg_ids2_0')['predict_proba'].max()
x

In [ ]:
df['final_prediction'] = df.apply(lambda row: row['predict_proba'] if x[row['msg_ids2_0']] == row['predict_proba'] else 0, axis=1)
df['final_prediction'] = df['final_prediction'].apply(round)

In [ ]:
df['final_prediction'] 

In [ ]:
metrics = precision_recall_fscore_support(df['label'], (df['final_prediction'] > 0.5))
metrics_df = pd.DataFrame.from_dict({'class': [0, 1],
                                     'precision': metrics[0],
                                     'recall': metrics[1],
                                     'f1': metrics[2],
                                     'support': metrics[3]})
metrics_df

In [ ]:
def init_predicted_dialogs(df):
    predicted_dialogs = {}
    known_df = df[df['is_root'] == 1]
    for tuple_ in known_df.itertuples():
        predicted_dialogs[tuple_.id] = tuple_.dialog_number
    return predicted_dialogs

In [ ]:
initial_labels = init_predicted_dialogs(original)
initial_labels

In [ ]:
def find_set_index(sets, id_):
    for i, set_ in enumerate(sets):
        if id_ in set_:
            return i
    return -1


def join_sets(sets, first_index, second_index):
    if first_index == second_index:
        return
        
    for id_ in sets[second_index]:
        sets[first_index].add(id_)
    sets[second_index] = set()


def join(df):
    sets = [set(msg_ids) for msg_ids in list(df['msg_ids1']) + list(df['msg_ids2'])]
    sets = set([str(set_) for set_ in sets])
    sets = [eval(set_) for set_ in sets]
    
    print("initial sets", len(sets), sets)
    for tuple_ in df.itertuples():
        if tuple_.final_prediction == 1:
            first_index = find_set_index(sets, tuple_.msg_ids1[0])
            second_index = find_set_index(sets, tuple_.msg_ids2[0])
            join_sets(sets, first_index, second_index)
    return [set_ for set_ in sets if len(set_) > 0]

In [ ]:
sets = join(df)
len(sets), sets

In [ ]:
df[df['msg_ids1_0'] == '3123'][['msg_ids1', 'msg_ids2', 'msgs_between', 'predict_proba', 'predict', 'final_prediction', 'label']]

In [ ]:
df[df['msg_ids2_0'] == '3123'][['msg_ids1', 'msg_ids2', 'msgs_between', 'predict_proba', 'predict', 'final_prediction', 'label']]

In [ ]:
sets

In [ ]:
from collections import defaultdict

In [ ]:
def define_label(set_, initial_labels):
    res = None
    for id_, label in initial_labels.items():
        if id_ in set_:
            if res is None:
                res = label
            else:
                return -1
    return res if res is not None else -1


def label_sets(sets, initial_labels):
    res = defaultdict(list)
    for set_ in sets:
        label = define_label(set_, initial_labels)
        res[label] += list(set_)
    return res

In [ ]:
final_labels = label_sets(sets, initial_labels)

In [ ]:
final_labels[1]

In [ ]:
final_labels[135]

In [ ]:
def find_label(id_, final_labels):
    for label, ids in final_labels.items():
        if id_ in ids:
            return label
    return -1

original['predicted_dialog_number'] = original['id'].apply(lambda id_: find_label(id_, final_labels))

In [ ]:
original.shape

In [ ]:
original[original['dialog_number'] == 1]

In [ ]:
should_predict_df = original[(original['should_predict'] == 1)]
true_predict_df = should_predict_df[(should_predict_df['predicted_dialog_number'] == should_predict_df['dialog_number'])]

accuracy = true_predict_df.shape[0] / should_predict_df.shape[0]
print(f'Accuracy: {accuracy}', f'{true_predict_df.shape[0]}, {should_predict_df.shape[0]}')

In [ ]:
should_predict_df[(should_predict_df['predicted_dialog_number'] != should_predict_df['dialog_number'])]

In [ ]:
print("without label: ", 
      should_predict_df[should_predict_df['predicted_dialog_number'] == -1].shape[0] / should_predict_df.shape[0])

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
df[df['msg_ids2_0'] == '3112__3113']

In [ ]:
df[['msg_ids1', 'msg_ids2', 'msgs_between', 'predict_proba', 'predict', 'final_prediction', 'label']].head(100)

In [ ]:
df[df['label'] == 1]['msgs_between'].value_counts()

In [ ]:
df[df['label'] == 0]['msgs_between'].value_counts()